# btlflt.ipynb

In [24]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display, clear_output
from icecream import ic

In [25]:
def calc_intensity_list(images_dir, first_frame_num, last_frame_num, normalize=True):
    """
    Calculates the average intensity of pixels within each frame of the video scene.
    If normalize is True, intensity_list will be normalized so that mean is zero and range is -1 to 1.
    """
    intensity_list = []
    for frame_num in range(first_frame_num, last_frame_num + 1):
        image = cv2.imread(f'{images_dir}/{frame_num:04d}.png')
        intensity = np.mean(image)
        intensity_list.append(intensity)
    if normalize:
        intensity_list = normalize_array(intensity_list)
    return intensity_list

# calc_intensity_list(FRAMES_DIR, FIRST_FRAME_NUM, LAST_FRAME_NUM)

In [26]:
def create_frame_intensity_figure(frame_num, seconds_list, intensity_list, frame_path, fig_path):
    """
    Creates an image with two subplots: the frame image and the intensity plot.
    """
 
    # Create figure
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6,6))

    # Create image plot
    image = plt.imread(frame_path)
    ax1.imshow(image)
    ax1.set_title(f'Frame {frame_num} | normalized mean pixel intensity: {intensity_list[frame_num-1]:.2f}')
    ax1.axis('off')

    # Create a sample plot
    x = seconds_list[:frame_num]
    y = intensity_list[:frame_num]

    ax2.plot(x, y)
    ax2.set_xlabel('time (s)')
    ax2.set_ylabel('normalized mean pixel intensity')
    ax2.set_xlim(min(seconds_list), max(seconds_list))
    ax2.set_ylim(-1, 1)

    # plt.tight_layout()
    
    # Save figure in a file
    fig.tight_layout
    fig.savefig(fig_path)
    plt.close(fig)
 
   
# create_frame_intensity_figure(1, seconds_list, intensity_list, 'frames/0001.png', 'test.png')

In [27]:
def normalize_array(arr):
    """ 
    Normalize array to range between -1 and 1 with a mean of 0.
    """ 
    arr = arr - np.mean(arr)
    scale_factor = max(abs(np.min(arr)), np.max(arr))
    return arr / scale_factor

# normalized_intensity_list = normalize_array(intensity_list)

In [28]:
# Note: ```.gitignore``` includes VIDEOPATH and FRAMESDIR

VIDEO_PATH = 'video.mp4'
FRAMES_DIR = 'frames'
FIGURES_DIR = 'figures'
FIRST_FRAME_NUM = 1
LAST_FRAME_NUM = 718
FPS = 6000

In [29]:
# Step 1: Download the video (best quality, no audio)

if os.path.exists(VIDEO_PATH):
    pass
else:
    command = f'yt-dlp -o {VIDEO_PATH} -f bestvideo https://www.youtube.com/watch?v=IUQSZHr9h9Q'
    os.system(command)

In [30]:
# Step 2: Extract frames from the video

if os.path.exists(FRAMES_DIR):
    pass
else:
    os.mkdir(FRAMES_DIR)
    command = f'ffmpeg -r 1 -i {VIDEO_PATH} -r 1 {FRAMES_DIR}/%04d.png'
    os.system(command)

In [31]:
# Step 3: Create a plot of intensity vs time

# create seconds_list which will be used for the x axis
seconds_list = [i/FPS for i in range(LAST_FRAME_NUM)]

# create intensity_list which will be used for the y axis
intensity_list = calc_intensity_list(FRAMES_DIR, FIRST_FRAME_NUM, LAST_FRAME_NUM)

# ensure that FIGURES_DIR exists
os.makedirs(FIGURES_DIR, exist_ok=True)

for frame_num in range(FIRST_FRAME_NUM, LAST_FRAME_NUM + 1):
    create_frame_intensity_figure(
        frame_num,
        seconds_list, 
        intensity_list, 
        frame_path = f'{FRAMES_DIR}/{frame_num:04d}.png', 
        fig_path = f'{FIGURES_DIR}/{frame_num:04d}.png'
        )

In [32]:
print('FINISHED')

FINISHED
